In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [8]:
data = pd.read_csv('Churn_modelling.csv')
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encoder_gender= LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo= OneHotEncoder(handle_unknown='ignore')
geo_encoded= onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)



In [9]:
## define a function to create the model
def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [19]:
## create a KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=1,layers=1, neurons=32)


In [20]:
pram_grid = {
    'neurons': [32, 64, 128],  
    'layers': [1, 2, 3],
    'epochs': [50, 100]}

In [21]:
grid= GridSearchCV(estimator=model, param_grid=pram_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

c:\Users\ASUS\anaconda3\envs\venv\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100


250/250 [==============================] - 1s 785us/step - loss: 0.5173 - accuracy: 0.7582
Epoch 2/100
250/250 [==============================] - 0s 762us/step - loss: 0.4217 - accuracy: 0.8189
Epoch 3/100
250/250 [==============================] - 0s 743us/step - loss: 0.4020 - accuracy: 0.8319
Epoch 4/100
250/250 [==============================] - 0s 734us/step - loss: 0.3845 - accuracy: 0.8416
Epoch 5/100
250/250 [==============================] - 0s 745us/step - loss: 0.3694 - accuracy: 0.8497
Epoch 6/100
250/250 [==============================] - 0s 737us/step - loss: 0.3585 - accuracy: 0.8540
Epoch 7/100
250/250 [==============================] - 0s 812us/step - loss: 0.3525 - accuracy: 0.8555
Epoch 8/100
250/250 [==============================] - 0s 742us/step - loss: 0.3479 - accuracy: 0.8591
Epoch 9/100
250/250 [==============================] - 0s 737us/step - loss: 0.3451 - accuracy: 0.8575
Epoch 10/100
250/250 [==============================] - 0s 743us/step -